In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfPred = dfPred.rename(columns=col_name)
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

#dfAll = dfAll.replace({'':np.nan,0:np.nan})
dfAll = dfAll.reset_index()

Load data...
Done.


In [ ]:
dfAll[pro_feat] = dfAll[pro_feat]-dfAll[pro_feat].mean()

In [ ]:
#dfAll = dfAll.replace({-99999999:-2.2222222})
dfAll[['750X1450','750X1451']]

In [ ]:
#dfAll = dfAll.replace({np.nan:-99999999})
pro_feat = [var for var in raw_predictors if not 'TOOL' in var ]
corr = dfAll[pro_feat].corr()
for i in range(len(corr)):
    corr.iloc[i,i:] =np.nan
print((corr==1.0).sum(axis=1))

In [ ]:
corr

In [ ]:
#处理0这个比较异常的值,在trian set中有40条数据，在某些column上为0 ，需要处理，test_a中也有一条
feature = ['ID']
for var in raw_predictors:
    if 'TOOL' in var:
        feature.append(var)
pick_list = (((corr==1.0).sum(axis=1))==0).values
for i in range(len(pick_list)):
    if pick_list[i]==True:
        feature.append(corr.index[i])
print(len(feature))

In [ ]:
#number of NAN
dfAll['Total_NAN_Num']=dfAll[raw_predictors].T.isnull().sum().tolist()
for key,pro in procedure.items():
    dfAll[key+'_NAN_Num']=dfAll[pro].T.isnull().sum().tolist()

In [ ]:
#Count 计算特定值在变量中重复出现的次数'''
dfAll = dfAll.replace({np.nan:-99999999})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    dfAll['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dfAll.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dfAll['cnt_'+var].ix[group.index] = count
dfAll[added].to_csv('../../Cache/Feat_cnt_col.csv',index=False)

added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
dfAll[added].to_csv('../../Cache/Feat_cnt_tool.csv',index=False)

In [20]:
#percentile 计算某一个值在全集中的位置（百分制）
dfAll = dfAll.replace({-99999999:np.nan})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    if 'TOOL' not in var:
        dfAll['pcent_'+var] = dfAll[var].rank(method='max')/float(len(dfAll))
        added.append('pcent_'+var)
dfAll[added].to_csv('../../Cache/Feat_pcent_col.csv',index=False)
t = 0
added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
        for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
            
dfAll[added].to_csv('../../Cache/Feat_pcent_tool.csv',index=False) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
#Pivot 创建多组category variable 的两两交叉表,感觉只能用来计算分类型变量,好像现在直接可以用pandas解决
def get_pivottable(X_train, X_test, use='all',Feat_list):
    """
    Returns a list of dictionaries, one per feature in the
    basic data, containing cross-tabulated counts
    for each column and each value of the feature.
    """
    dictionaries = []
    if use == 'all':
        X = np.vstack((X_train, X_test))
        filename = "pivottable"
    elif use == 'train':
        X = X_train
        filename = "pivottable_train"
    else:
        X = X_test
        filename = "pivottable_test"
    for i in range(len(Feat_list)):
        dictionaries.append({'total': 0})
    try:
        with open("cache/%s.pkl" % filename, 'rb') as f:
            logger.debug("loading cross-tabulated data from cache")
            dictionaries = pickle.load(f)
    except IOError:
        logger.debug("no cache found, cross-tabulating data")
        for i, row in enumerate(X):
            for j in range(len(Feat_list)):
                dictionaries[j]['total'] += 1
                if row[j] not in dictionaries[j]:
                    dictionaries[j][row[j]] = {'total': 1}
                    for k, key in enumerate(Feat_list):
                        dictionaries[j][row[j]][key] = {row[k]: 1}
                else:
                    dictionaries[j][row[j]]['total'] += 1
                    for k, key in enumerate(Feat_list):
                        if row[k] not in dictionaries[j][row[j]][key]:
                            dictionaries[j][row[j]][key][row[k]] = 1
                        else:
                            dictionaries[j][row[j]][key][row[k]] += 1
        with open("cache/%s.pkl" % filename, 'wb') as f:
            pickle.dump(dictionaries, f, pickle.HIGHEST_PROTOCOL)

    return dictionaries

def creat_cate_corr(X_train, X_test, Feat_list=[var if 'TOOL' in var for var in raw_predictor]):
    features_train = []
    features_test = []
    dictionaries = get_pivottable(X_train[Feat_list], X_test[Feat_list],Feat_list=Feat_list)
    #dictionaries_train = get_pivottable(X_train[Feat_list], X_test[Feat_list], use='train',Feat_list=Feat_list)
    #dictionaries_test = get_pivottable(X_test[Feat_list], X_test[Feat_list], use='test',Feat_list=Feat_list)

In [40]:
dfAll = pd.read_csv('../../Data/All_Data_dedup.csv')
feat_needed = dfAll.columns.tolist()[14:]
'''gap_list = []
for i in range(len(feat_needed)-1):
    for j in range(i+1,len(feat_needed)):
        if dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)<=1 and dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)>=-1:
            tmp_var = feat_needed[i]+'-'+feat_needed[j]
            dfAll[tmp_var] = dfAll[feat_needed[i]]- dfAll[feat_needed[j]]
            gap_list.append(tmp_var)
            print(tmp_var)'''

"gap_list = []\nfor i in range(len(feat_needed)-1):\n    for j in range(i+1,len(feat_needed)):\n        if dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)<=1 and dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)>=-1:\n            tmp_var = feat_needed[i]+'-'+feat_needed[j]\n            dfAll[tmp_var] = dfAll[feat_needed[i]]- dfAll[feat_needed[j]]\n            gap_list.append(tmp_var)\n            print(tmp_var)"

In [41]:
feat_needed2 =[]
for var in feat_needed:
    if dfAll[var].nunique()>15:
        feat_needed2.append(var)
gap_list = []
for i in range(len(feat_needed2)-1):
    for j in range(i+1,len(feat_needed2)):
        if dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)<=2 and dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)>=-2:
            tmp_var = feat_needed2[i]+'-'+feat_needed2[j]
            dfAll[tmp_var] = dfAll[feat_needed2[i]]- dfAll[feat_needed2[j]]
            gap_list.append(tmp_var)
            print(tmp_var)

210X1-220X390
210X1-311X219
210X1-261X228
210X1-261X238
210X1-261X247
210X1-261X257
210X1-261X268
210X1-261X289
210X1-261X414
210X1-261X424
210X1-261X433
210X1-261X443
210X1-261X454
210X1-261X463
210X1-261X591
210X1-261X599
210X1-261X610
210X1-261X619
210X1-261X629
210X1-261X640
210X1-261X649
210X1-261X661
210X1-312X714
210X1-312X720
210X1-312X723
210X1-312X726
210X1-312X776
210X1-312X777
210X1-312X778
210X1-312X779
210X1-312X780
210X1-312X782
210X1-312X783
210X1-312X784
210X1-312X785
210X1-312X786
210X1-312X788
210X1-312X789
210X1-312X790
210X1-312X791
210X1-312X792
210X1-330X81
210X1-330X113
210X1-330X120
210X1-330X127
210X1-330X128
210X1-330X134
210X1-330X141
210X1-330X277
210X1-330X287
210X1-330X304
210X1-330X314
210X1-330X324
210X1-330X334
210X1-330X354
210X1-330X364
210X1-330X461
210X1-330X481
210X1-330X491
210X1-330X492
210X1-330X501
210X1-330X502
210X1-330X508
210X1-330X518
210X1-330X519
210X1-330X528
210X1-330X529
210X1-330X538
210X1-340X24
210X1-340X25
210X1-340X26
210X1-340X

210X4-311X206
210X4-261X343
210X4-261X377
210X4-261X378
210X4-261X379
210X4-261X381
210X4-261X382
210X4-261X383
210X4-261X385
210X4-261X387
210X4-261X529
210X4-261X563
210X4-261X564
210X4-261X565
210X4-261X567
210X4-261X568
210X4-261X569
210X4-261X571
210X4-261X572
210X4-261X573
210X4-261X715
210X4-261X749
210X4-261X750
210X4-261X751
210X4-261X753
210X4-261X754
210X4-261X755
210X4-261X757
210X4-261X758
210X4-261X759
210X4-312X69
210X4-312X70
210X4-312X71
210X4-312X72
210X4-312X76
210X4-312X77
210X4-312X78
210X4-312X81
210X4-312X82
210X4-312X83
210X4-312X84
210X4-312X694
210X4-330X42
210X4-330X43
210X4-330X49
210X4-330X52
210X4-330X53
210X4-330X59
210X4-330X61
210X4-330X103
210X4-330X527
210X4-330X537
210X4-330X546
210X4-330X689
210X4-330X699
210X4-330X798
210X4-330X896
210X4-330X1031
210X4-330X1033
210X4-330X1036
210X4-340X17
210X4-340X19
210X4-344X33
210X4-344X34
210X4-344X37
210X4-344X39
210X4-344X40
210X4-344X309
210X4-360X46
210X4-360X57
210X4-360X582
210X4-360X763
210X4-360X1139
2

210X8-210X228
210X8-210X230
210X8-210X231
210X8-220X2
210X8-220X4
210X8-220X5
210X8-220X6
210X8-220X8
210X8-220X9
210X8-220X10
210X8-220X11
210X8-220X12
210X8-220X13
210X8-220X110
210X8-220X114
210X8-220X118
210X8-220X122
210X8-220X126
210X8-220X130
210X8-220X163
210X8-220X164
210X8-220X165
210X8-220X166
210X8-220X167
210X8-220X168
210X8-220X169
210X8-220X171
210X8-220X172
210X8-220X174
210X8-220X176
210X8-220X204
210X8-220X216
210X8-220X217
210X8-220X218
210X8-220X289
210X8-220X484
210X8-220X490
210X8-220X491
210X8-220X492
210X8-220X493
210X8-220X494
210X8-220X495
210X8-220X497
210X8-220X498
210X8-220X499
210X8-220X500
210X8-220X501
210X8-220X502
210X8-220X503
210X8-220X504
210X8-220X505
210X8-220X506
210X8-220X507
210X8-220X508
210X8-220X509
210X8-220X510
210X8-220X512
210X8-220X514
210X8-220X515
210X8-220X517
210X8-220X520
210X8-220X526
210X8-220X550
210X8-310X2
210X8-310X4
210X8-310X5
210X8-310X6
210X8-310X8
210X8-310X9
210X8-310X10
210X8-310X11
210X8-310X12
210X8-310X13
210X8-310X

210X12-220X289
210X12-220X490
210X12-220X491
210X12-220X492
210X12-220X493
210X12-220X494
210X12-220X495
210X12-220X497
210X12-220X498
210X12-220X499
210X12-220X500
210X12-220X501
210X12-220X502
210X12-220X503
210X12-220X504
210X12-220X505
210X12-220X506
210X12-220X507
210X12-220X508
210X12-220X509
210X12-220X510
210X12-220X512
210X12-220X514
210X12-220X515
210X12-220X517
210X12-220X520
210X12-220X526
210X12-220X550
210X12-310X2
210X12-310X4
210X12-310X5
210X12-310X6
210X12-310X8
210X12-310X9
210X12-310X10
210X12-310X11
210X12-310X13
210X12-310X99
210X12-310X103
210X12-310X107
210X12-310X111
210X12-310X115
210X12-310X119
210X12-310X147
210X12-310X148
210X12-310X149
210X12-310X150
210X12-310X151
210X12-310X152
210X12-310X153
210X12-310X154
210X12-310X155
210X12-310X156
210X12-310X157
210X12-310X158
210X12-310X159
210X12-310X160
210X12-310X171
210X12-310X174
210X12-310X175
210X12-310X176
210X12-310X181
210X12-310X188
210X12-310X200
210X12-310X201
210X12-310X202
210X12-311X33
210X12-311X6

210X14-220X492
210X14-220X493
210X14-220X494
210X14-220X495
210X14-220X497
210X14-220X498
210X14-220X499
210X14-220X500
210X14-220X501
210X14-220X502
210X14-220X503
210X14-220X504
210X14-220X505
210X14-220X506
210X14-220X507
210X14-220X508
210X14-220X509
210X14-220X510
210X14-220X512
210X14-220X514
210X14-220X515
210X14-220X517
210X14-220X520
210X14-220X526
210X14-220X550
210X14-310X2
210X14-310X4
210X14-310X5
210X14-310X6
210X14-310X8
210X14-310X9
210X14-310X10
210X14-310X11
210X14-310X12
210X14-310X13
210X14-310X99
210X14-310X103
210X14-310X107
210X14-310X111
210X14-310X115
210X14-310X119
210X14-310X147
210X14-310X148
210X14-310X149
210X14-310X150
210X14-310X151
210X14-310X152
210X14-310X153
210X14-310X154
210X14-310X155
210X14-310X156
210X14-310X157
210X14-310X158
210X14-310X159
210X14-310X160
210X14-310X171
210X14-310X174
210X14-310X175
210X14-310X176
210X14-310X181
210X14-310X188
210X14-310X200
210X14-310X201
210X14-310X202
210X14-311X33
210X14-311X69
210X14-311X103
210X14-311X104

...

750X116-750X583
750X116-750X594
750X116-750X604
750X116-750X611
750X116-750X612
750X116-750X620
750X116-750X628
750X116-750X636
750X116-750X644
750X116-750X652
750X116-750X659
750X116-750X660
750X116-750X668
750X116-750X676
750X116-750X745
750X116-750X778
750X116-750X789
750X116-750X1192
750X116-750X1247
750X118-750X124
750X118-750X131
750X118-750X132
750X118-750X140
750X118-750X148
750X118-750X156
750X118-750X164
750X118-750X175
750X118-750X185
750X118-750X186
750X118-750X197
750X118-750X210
750X118-750X218
750X118-750X229
750X118-750X240
750X118-750X251
750X118-750X262
750X118-750X273
750X118-750X284
750X118-750X295
750X118-750X306
750X118-750X317
750X118-750X328
750X118-750X335
750X118-750X336
750X118-750X347
750X118-750X358
750X118-750X369
750X118-750X380
750X118-750X391
750X118-750X402
750X118-750X413
750X118-750X424
750X118-750X435
750X118-750X446
750X118-750X454
750X118-750X465
750X118-750X476
750X118-750X487
750X118-750X498
750X118-750X509
750X118-750X520
750X118-750X531
750X11

750X164-750X789
750X164-750X1192
750X164-750X1247
750X175-750X185
750X175-750X186
750X175-750X197
750X175-750X210
750X175-750X218
750X175-750X229
750X175-750X240
750X175-750X251
750X175-750X262
750X175-750X273
750X175-750X284
750X175-750X295
750X175-750X306
750X175-750X317
750X175-750X328
750X175-750X335
750X175-750X336
750X175-750X347
750X175-750X358
750X175-750X369
750X175-750X380
750X175-750X391
750X175-750X402
750X175-750X413
750X175-750X424
750X175-750X435
750X175-750X446
750X175-750X454
750X175-750X465
750X175-750X476
750X175-750X487
750X175-750X498
750X175-750X509
750X175-750X520
750X175-750X531
750X175-750X542
750X175-750X553
750X175-750X564
750X175-750X572
750X175-750X583
750X175-750X594
750X175-750X604
750X175-750X611
750X175-750X612
750X175-750X620
750X175-750X628
750X175-750X636
750X175-750X644
750X175-750X652
750X175-750X659
750X175-750X660
750X175-750X668
750X175-750X676
750X175-750X745
750X175-750X778
750X175-750X789
750X175-750X1192
750X175-750X1247
750X185-750X186
750X

750X262-750X531
750X262-750X542
750X262-750X553
750X262-750X564
750X262-750X572
750X262-750X583
750X262-750X594
750X262-750X604
750X262-750X611
750X262-750X612
750X262-750X620
750X262-750X628
750X262-750X636
750X262-750X644
750X262-750X652
750X262-750X659
750X262-750X660
750X262-750X668
750X262-750X676
750X262-750X745
750X262-750X778
750X262-750X789
750X262-750X1192
750X262-750X1247
750X273-750X284
750X273-750X295
750X273-750X306
750X273-750X317
750X273-750X328
750X273-750X335
750X273-750X336
750X273-750X347
750X273-750X358
750X273-750X369
750X273-750X380
750X273-750X391
750X273-750X402
750X273-750X413
750X273-750X424
750X273-750X435
750X273-750X446
750X273-750X454
750X273-750X465
750X273-750X476
750X273-750X487
750X273-750X498
750X273-750X509
750X273-750X520
750X273-750X531
750X273-750X542
750X273-750X553
750X273-750X564
750X273-750X572
750X273-750X583
750X273-750X594
750X273-750X604
750X273-750X611
750X273-750X612
750X273-750X620
750X273-750X628
750X273-750X636
750X273-750X644
750X27

750X380-750X542
750X380-750X553
750X380-750X564
750X380-750X572
750X380-750X583
750X380-750X594
750X380-750X604
750X380-750X611
750X380-750X612
750X380-750X620
750X380-750X628
750X380-750X636
750X380-750X644
750X380-750X652
750X380-750X659
750X380-750X660
750X380-750X668
750X380-750X676
750X380-750X745
750X380-750X778
750X380-750X789
750X380-750X1192
750X380-750X1247
750X391-750X402
750X391-750X413
750X391-750X424
750X391-750X435
750X391-750X446
750X391-750X454
750X391-750X465
750X391-750X476
750X391-750X487
750X391-750X498
750X391-750X509
750X391-750X520
750X391-750X531
750X391-750X542
750X391-750X553
750X391-750X564
750X391-750X572
750X391-750X583
750X391-750X594
750X391-750X604
750X391-750X611
750X391-750X612
750X391-750X620
750X391-750X628
750X391-750X636
750X391-750X644
750X391-750X652
750X391-750X659
750X391-750X660
750X391-750X668
750X391-750X676
750X391-750X745
750X391-750X778
750X391-750X789
750X391-750X1192
750X391-750X1247
750X402-750X413
750X402-750X424
750X402-750X435
750X

750X572-750X668
750X572-750X676
750X572-750X745
750X572-750X778
750X572-750X789
750X572-750X1192
750X572-750X1247
750X582-750X741
750X582-750X763
750X582-750X778
750X583-750X594
750X583-750X604
750X583-750X611
750X583-750X612
750X583-750X620
750X583-750X628
750X583-750X636
750X583-750X644
750X583-750X652
750X583-750X659
750X583-750X660
750X583-750X668
750X583-750X676
750X583-750X745
750X583-750X778
750X583-750X789
750X583-750X1192
750X583-750X1247
750X594-750X604
750X594-750X611
750X594-750X612
750X594-750X620
750X594-750X628
750X594-750X636
750X594-750X644
750X594-750X652
750X594-750X659
750X594-750X660
750X594-750X668
750X594-750X676
750X594-750X745
750X594-750X778
750X594-750X789
750X594-750X1192
750X594-750X1247
750X604-750X611
750X604-750X612
750X604-750X620
750X604-750X628
750X604-750X636
750X604-750X644
750X604-750X652
750X604-750X659
750X604-750X660
750X604-750X668
750X604-750X676
750X604-750X745
750X604-750X778
750X604-750X789
750X604-750X1192
750X604-750X1247
750X611-750X612


In [48]:
dfAll = dfAll.set_index(dfAll['ID'])
dfTrain_tmp = dfAll.loc[dfTrain['ID']]
dfPred_tmp = dfAll.loc[dfPred['ID']]

In [51]:
dfPred = dfPred_tmp.copy(deep=True)
dfTrain = dfTrain_tmp.copy(deep=True)

In [52]:
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [53]:
dfAll2 = pd.read_csv('../../Data/All_Data_dedup.csv',nrows=2)
raw_predictors = dfAll2.columns[1:]

In [55]:
dfTrain.columns

Index(['ID', 'TOOL_210', 'TOOL_220', 'TOOL_300', 'TOOL_310', 'TOOL_311',
       'TOOL_312', 'TOOL_330', 'TOOL_340', 'TOOL_344',
       ...
       'pcent_Pro_750_750X1347', 'pcent_Pro_750_750X1350',
       'pcent_Pro_750_750X1357', 'pcent_Pro_750_750X1370',
       'pcent_Pro_750_750X1380', 'pcent_Pro_750_750X1384',
       'pcent_Pro_750_750X1398', 'pcent_Pro_750_750X1408',
       'pcent_Pro_750_750X1442', 'pcent_Pro_750_750X1452'],
      dtype='object', length=102084)

In [57]:
f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()
predictors = dfTrain.columns.tolist()[1:]+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        try:
             predictors.remove(var)
        except:
            continue
    for var in var_change[key]['category']:
        try:
            predictors.remove(var)
        except:
            continue
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [61]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.4}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    #print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [59]:
dfTrain2 = pd.read_csv(config.original_train_data_path,usecols=['ID','Y'])
dfTrain = dfTrain.merge(dfTrain2,'inner','ID')
dfTrain.shape

(500, 106207)

In [60]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33634	eval-rmse:2.31181
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[676]	train-rmse:0.048391	eval-rmse:0.16906

[0]	train-rmse:2.33684	eval-rmse:2.30968
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[459]	train-rmse:0.076249	eval-rmse:0.18978

[0]	train-rmse:2.3238	eval-rmse:2.36199
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[897]	train-rmse:0.033219	eval-rmse:0.178773

[0]	train-rmse:2.3339	eval-rmse:2.32154
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[588]	train-rmse:0.057792	

NameError: name 'metrics' is not defined

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)
imp.to_csv('../../Profiling/imp_test_v3_%s.csv'%today,index=False)
test_result('../../Profiling/test_score_%s.csv'%today,index=False)